In [73]:
# Import Modules
%load_ext autoreload
%autoreload 2

from math import pi
import pandas as pd
import sys
import numpy as np
from pathlib import Path
from dotenv import load_dotenv

import bokeh
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import HoverTool

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Set paths

PROJECT_ROOT = !git rev-parse --show-toplevel
PROJECT_ROOT = Path(PROJECT_ROOT[0])
print(PROJECT_ROOT)
sys.path.append(str(PROJECT_ROOT))
sys.path.append("/workspaces/template.python.project/tests")

/workspaces/template.python.project


In [3]:
from mock_dataset import mock_dataset

In [59]:
specs = {"float": [200, 1, 0] \
                        ,"int": [200, 1, 0] \
                        ,"categorical": [200, 1, 0] \
                        ,"bool": [200, 1, 0] \
                        ,"str": [200, 1, 0] \
                        ,"datetime": [200, 1, 0] \
                        }
df = mock_dataset(specs=specs)


In [60]:
df

,float_0,int_0,categorical_0,bool_0,str_0,datetime_0
0,0.169534,142673.0,DCA,1.0,FJMUHMPPOLBVUYQDQQKMWRRUDQBEOGKBIPRCBOSCPLBJZR...,2021-02-03 11:36:26.648699
1,0.397219,423732.0,DBD,1.0,DIASHXVVTUPGPCYNTCCUNIICPLQNIABDFOFWYMHDLUKYPH...,2021-02-04 11:36:26.648699
2,0.007835,200127.0,BAA,1.0,JTESAQAXCKYSGMQONEALSRWJLDSIUSYFIGTDDKMUWWYOYH...,2021-02-05 11:36:26.648699
3,0.970437,65292.0,CAC,1.0,AMRYYDCJMFWJPFGZQHVVWYDEPFBCPGRIYJHNLNUJLYZVYF...,2021-02-06 11:36:26.648699
4,0.933498,71068.0,BCC,1.0,TDIYZTTGZAGXHENDBMGHAERTSMLKKDVLCEHREMSLFGNHDN...,2021-02-07 11:36:26.648699
...,...,...,...,...,...,...
195,0.999799,390522.0,BCB,0.0,BAQYNEELDMFRNYDVSOLPSQWZQSVQUEBFTQZAQPOJPXMFDH...,2021-08-17 11:36:26.648699
196,0.174346,380361.0,ABC,1.0,MBJEIOXCCKDUHQNIXNNVNFRBZBUYISNGIYUIWTVBYZQSWJ...,2021-08-18 11:36:26.648699
197,0.048045,291541.0,ABA,1.0,SMJHOMJYKMVNVJZAQBLEXXGJECKCYKCEUCVORSVVYVVCOS...,2021-08-19 11:36:26.648699
198,0.204184,123440.0,DCD,1.0,VQCHLUXFBVMTCJZSLPGPSFYQPDYKGCTTVAQCIUHAPBBMSI...,2021-08-20 11:36:26.648699


# Line Plot

In [113]:
# src: https://stackoverflow.com/questions/61792141/select-and-update-pandas-dataframe-columns-in-bokeh-plot
#!TODO: FIX ISSUE WITH HOVER

from bokeh.layouts import column
from bokeh.models import ColumnDataSource, CustomJS, Select, TextInput

output_notebook()

ds = ColumnDataSource(df.select_dtypes(include=[np.number, "datetime64[ns]"]))

x = "datetime_0"
y = "int_0"

# create a plot and style its properties
line = figure(plot_width=1000, plot_height=400, title=f"{y} vs {x}", toolbar_location="above")
line.yaxis.axis_label = y
line.xaxis.axis_label = x

# Format the tooltip
tooltips = [(y, f"@{y}"), (x, f"@{x}")]

# Add the HoverTool to the figure
hover = HoverTool(tooltips=tooltips)

line.add_tools(HoverTool(tooltips=tooltips))

# create line renderer
line_renderer = line.line(x=x, y=y, line_width=2, source=ds)
circle_renderer = line.circle(x=x, y=y, source=ds)

line.grid.grid_line_alpha=0.3

text_input = TextInput(title="Add graph title", value='')
handler = CustomJS(args=dict(line_renderer=line_renderer, circle_renderer=circle_renderer, 'title': fig.title, 'text_input': text_input), code="""
   line_renderer.glyph.y = {field: cb_obj.value};
   circle_renderer.glyph.y = {field: cb_obj.value};
   title.text = text_input.value
""")

select = Select(title="Column", options=list(df.select_dtypes(include=[np.number]).columns))
select.js_on_change('value', handler)

show(column(select, line))

Loading BokehJS ...

# Bar Plot

In [65]:
output_notebook()

x = "datetime_0"
y = "int_0"

bar = figure(plot_width=1000, plot_height=400, title=f"{y} vs {x}")

bar.vbar(x=x, top=y, source=df)

# Format the tooltip
tooltips = [(y, f"@{y}"), (x, f"@{x}")]

# Configure a renderer to be used upon hover
hover_glyph = bar.circle(x=x, y=y, source=df, size=15, alpha=0, hover_fill_color='black', hover_alpha=0.5)

# Add the HoverTool to the figure
bar.add_tools(HoverTool(tooltips=tooltips, renderers=[hover_glyph]))

bar.grid.grid_line_alpha=0.3

show(bar)

Loading BokehJS ...

# Combine Line and Bar Plots

In [56]:
output_notebook()

x = "datetime_0"
y_line = "float_0"
y_bar = "int_0"

# create a plot and style its properties
line = figure(plot_width=1000, plot_height=400, title=f"{y_line} vs {x}")
line.yaxis.axis_label = y_line

# create line renderer
line.line(x=x, y=y_line, line_width=2, source=df)
line.circle(x=x, y=y_line, source=df)

line.grid.visible = False
line.xaxis.visible = False

# Format the tooltip
tooltips = [(y_line, f"@{y_line}"), (x, f"@{x}")]

# Configure a renderer to be used upon hover
hover_glyph = line.circle(x=x, y=y_line, source=df, size=15, alpha=0, hover_fill_color='black', hover_alpha=0.5)

# Add the HoverTool to the figure
line.add_tools(HoverTool(tooltips=tooltips, renderers=[hover_glyph]))

bar = figure(plot_width=1000, plot_height=400, title=f"{y_bar} vs {x}", x_range=line.x_range)

bar.vbar(x=x, top=y, source=df)

# Format the tooltip
tooltips = [(y_bar, f"@{y_bar}"), (x, f"@{x}")]

# Configure a renderer to be used upon hover
hover_glyph = bar.circle(x=x, y=y_bar, source=df, size=15, alpha=0, hover_fill_color='black', hover_alpha=0.5)

# Add the HoverTool to the figure
bar.add_tools(HoverTool(tooltips=tooltips, renderers=[hover_glyph]))

# 
bar.yaxis.axis_label = y_bar
bar.xaxis.axis_label = x
bar.grid.visible = False


show(gridplot([[line],[bar]]))

Loading BokehJS ...

In [57]:
output_notebook()

from bokeh.io import show
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, CustomJS
from bokeh.models import Select
from bokeh.models.tools import HoverTool
from bokeh.plotting import figure

df = pd.DataFrame({'2007': [10, 20, 30, 40],
                   '2008': [90, 60, 70, 40],
                   '2009': [30, 60, 70, 10],
                   '2010': [80, 50, 30, 10]},
                  index=[0, 0.5, 1, 1.5])
ds = ColumnDataSource(df)
p = figure(toolbar_location="above", x_axis_type="linear")
p.add_tools(HoverTool(tooltips=[("y", "@index")]))

line_renderer = p.line('2007', 'index', source=ds)

handler = CustomJS(args=dict(line_renderer=line_renderer), code="""
   line_renderer.glyph.x = {field: cb_obj.value};
""")

select = Select(title="df-column:", options=list(df.columns))
select.js_on_change('value', handler)

show(column(select, p))


Loading BokehJS ...